In [1]:
from dotenv import load_dotenv
load_dotenv() # load environment variables

False

In [2]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF

In [3]:
# Set up logging and environment variables for connecting to the Neo4j database

# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

# Neo4j connection parameters
# Make sure Neo4j Desktop is running with a local DBMS started
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', "password")

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError('NEO4J_URI, NEO4J_USER, and NEO4J_PASSWORD must be set')

In [4]:
# Configure Graphiti for Ollama

from graphiti_core import Graphiti
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.llm_client.openai_client import OpenAIClient
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient

# Configure Ollama LLM client
llm_config = LLMConfig(
    api_key="abc",  # Ollama doesn't require a real API key
    model="gemma3n",
    small_model="gemma3n",
    base_url="http://localhost:11434/v1",  # Ollama provides this port
)
llm_client = OpenAIClient(config=llm_config)

# Initialize Graphiti with Ollama clients
graphiti = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
    llm_client=llm_client,
    embedder=OpenAIEmbedder(
        config=OpenAIEmbedderConfig(
            api_key="abc",
            embedding_model="nomic-embed-text",
            embedding_dim=768,
            base_url="http://localhost:11434/v1",
        )
    ),
    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

# Initialize the graph database with graphiti's indices. This only needs to be done once.
graphiti.build_indices_and_constraints()

<coroutine object Graphiti.build_indices_and_constraints at 0x107e3b030>

In [5]:
episodes = [
    {
        'content': 'Kamala Harris is the Attorney General of California. She was previously '
        'the district attorney for San Francisco.',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': 'As AG, Harris was in office from January 3, 2011 – January 3, 2017',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'state': 'California',
            'previous_role': 'Lieutenant Governor',
            'previous_location': 'San Francisco',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'term_start': 'January 7, 2019',
            'term_end': 'Present',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
]

# Add episodes to the graph
for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=f'Freakonomics Radio {i}',
        episode_body=episode['content']
        if isinstance(episode['content'], str)
        else json.dumps(episode['content']),
        source=episode['type'],
        source_description=episode['description'],
        reference_time=datetime.now(timezone.utc),
    )
    print(f'Added episode: Freakonomics Radio {i} ({episode["type"].value})')

2025-08-05 09:58:44 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:58:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:58:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:58:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:58:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:58:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:58:57 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:09 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:09 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Added episode: Freakonomics Radio 0 (text)


2025-08-05 09:59:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:40 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/complet

Added episode: Freakonomics Radio 1 (text)


2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 09:59:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:00:05 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:19 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:19 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Added episode: Freakonomics Radio 2 (json)


2025-08-05 10:00:35 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:35 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:00:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:51 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:51 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:00:53 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:53 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:00:55 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:00:57 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/

Added episode: Freakonomics Radio 3 (json)


In [6]:
# Basic Search

# Perform a hybrid search combining semantic similarity and BM25 retrieval
print("\nSearching for: 'Who was the California Attorney General?'")
results = await graphiti.search('Who was the California Attorney General?')

# Print search results
print('\nSearch Results:')
for result in results:
    print(f'UUID: {result.uuid}')
    print(f'Fact: {result.fact}')
    if hasattr(result, 'valid_at') and result.valid_at:
        print(f'Valid from: {result.valid_at}')
    if hasattr(result, 'invalid_at') and result.invalid_at:
        print(f'Valid until: {result.invalid_at}')
    print('---')


2025-08-05 10:00:59 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Searching for: 'Who was the California Attorney General?'

Search Results:
UUID: 014caa1b-2625-402a-9722-0ba7a81cc570
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 16:58:34.859349+00:00
---
UUID: cb211323-5154-4caf-9f39-9fa8b5f529a9
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 16:58:34.859349+00:00
Valid until: 2025-08-05 16:59:50.459896+00:00
---
UUID: 36304b1d-df2a-4ac6-9935-91b2c26e6c60
Fact: Gavin Newsom is the Governor of California.
Valid from: 2025-08-05 16:59:50.459896+00:00
---
UUID: 3aa53794-fb72-483b-87c2-816c47eab9f5
Fact: Gavin Newsom is the Governor of California.
Valid from: 2025-08-05 17:00:33.727681+00:00
---
UUID: 670b563a-7fda-4d67-ba60-a5b21cabadb9
Fact: She was previously the district attorney for San Francisco.
Valid from: 2025-08-05 16:58:34.859349+00:00
---
UUID: cd0c571b-06e3-4b1e-895e-8742a30389ed
Fact: She was previously the district attorney for San Francisco.
Valid from: 2025-08-05 16

In [7]:
# Center node search

# Use the top search result's UUID as the center node for reranking
if results and len(results) > 0:
    # Get the source node UUID from the top result
    center_node_uuid = results[0].source_node_uuid

    print('\nReranking search results based on graph distance:')
    print(f'Using center node UUID: {center_node_uuid}')

    reranked_results = await graphiti.search(
        'Who was the California Attorney General?', center_node_uuid=center_node_uuid
    )

    # Print reranked search results
    print('\nReranked Search Results:')
    for result in reranked_results:
        print(f'UUID: {result.uuid}')
        print(f'Fact: {result.fact}')
        if hasattr(result, 'valid_at') and result.valid_at:
            print(f'Valid from: {result.valid_at}')
        if hasattr(result, 'invalid_at') and result.invalid_at:
            print(f'Valid until: {result.invalid_at}')
        print('---')
else:
    print('No results found in the initial search to use as center node.')


2025-08-05 10:00:59 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Reranking search results based on graph distance:
Using center node UUID: 307c636b-c903-42a6-8c19-afc5f2d09e34

Reranked Search Results:
UUID: 014caa1b-2625-402a-9722-0ba7a81cc570
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 16:58:34.859349+00:00
---
UUID: 670b563a-7fda-4d67-ba60-a5b21cabadb9
Fact: She was previously the district attorney for San Francisco.
Valid from: 2025-08-05 16:58:34.859349+00:00
---
UUID: 9fd79e71-050a-4920-ac08-5642966cb081
Fact: As AG, Harris was in office from January 3, 2011 – January 3, 2017
Valid from: 2025-08-05 16:59:15.205131+00:00
Valid until: 2017-01-03 00:00:00+00:00
---
UUID: cb211323-5154-4caf-9f39-9fa8b5f529a9
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 16:58:34.859349+00:00
Valid until: 2025-08-05 16:59:50.459896+00:00
---
UUID: 53dd1b92-f7c8-4c22-ad7d-bbb87f6fec02
Fact: Gavin Newsom was formerly located in San Francisco.
Valid from: 2025-08-05 16:59:50.459896+00:00
---
UU

In [8]:
# Node Search Using Search Recipes

# Example: Perform a node search using _search method with standard recipes
print(
    '\nPerforming node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:'
)

# Use a predefined search configuration recipe and modify its limit
node_search_config = NODE_HYBRID_SEARCH_RRF.model_copy(deep=True)
node_search_config.limit = 5  # Limit to 5 results

# Execute the node search
node_search_results = await graphiti._search(
    query='California Governor',
    config=node_search_config,
)

# Print node search results
print('\nNode Search Results:')
for node in node_search_results.nodes:
    print(f'Node UUID: {node.uuid}')
    print(f'Node Name: {node.name}')
    node_summary = node.summary[:100] + '...' if len(node.summary) > 100 else node.summary
    print(f'Content Summary: {node_summary}')
    print(f"Node Labels: {', '.join(node.labels)}")
    print(f'Created At: {node.created_at}')
    if hasattr(node, 'attributes') and node.attributes:
        print('Attributes:')
        for key, value in node.attributes.items():
            print(f'  {key}: {value}')
    print('---')


2025-08-05 10:00:59 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Performing node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:

Node Search Results:
Node UUID: fb2de6f4-4e42-4635-a5d8-c64b53bb4fb6
Node Name: Governor
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 16:59:54.856541+00:00
Attributes:
  labels: ['Entity']
---
Node UUID: a54132df-ccaf-48b2-ab04-5d5996e0438e
Node Name: California
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 16:58:44.749366+00:00
Attributes:
  labels: ['Entity']
---
Node UUID: dc1db3a7-3ba4-4107-8e9b-73e68a6b0958
Node Name: Attorney General
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 16:58:44.749329+00:00
Attributes:
  labels: ['Entity']
---
Node 